In [1]:
import pandas as pd
import numpy as np
import scipy
from nltk.corpus import stopwords
import re
import os

In [74]:
## Loading data - an excell file :

data1 = pd.ExcelFile('Tranzact2.xlsx')
tranzact_data1 = data1.parse(0)

lamba = len(tranzact_data1)
print(lamba)
tranzact_data1.columns

28922


Index(['Company', 'Supplier', 'Item', 'Category', 'UOM', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')

In [92]:
tranzact_data2 = tranzact_data1.iloc[0:lamba, 0:3] ## Remove unnecessary columns

tranzact_data2['Item'] = tranzact_data2['Item'].map(lambda x: x if type(x)!=str else x.lower()) ## make Items in lower case


tranzact_data = tranzact_data2.dropna() ## remove rows with cell value none

tranzact_data = tranzact_data.reset_index(drop=True)
print(len(tranzact_data))

28921


In [93]:
## Cleaning the Item Description to create a feature matrix :

pd.options.mode.chained_assignment = None

tranzact_data['features'] = tranzact_data['Item'].str.replace(r'\b\d+\b','')  ## remove integers

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\W',' ')   ## remove puntuations

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\w\b','') ## remove stand alone single letters

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\s+',' ') ## remove gaps between words to singe gap

## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")

words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')

tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())


## Removing duplicate words from individual features

tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
                                                                                key=x.split().index))]).lower())

## Removing only single word features :

print(len(tranzact_data))
tranzact_data.tail()
print(tranzact_data['features'][28919].split())
print(len(tranzact_data['features'][28919].split()))
tranzact_data.tail()

28921
['ferrul', 'number']
2


,Company,Supplier,Item,features
28916,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'r',ferrul number
28917,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'y',ferrul number
28918,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'b',ferrul number
28919,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'n',ferrul number
28920,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long


In [98]:
tranzact_data.count()

Company     28921
Supplier    28921
Item        28921
features    28921
dtype: int64

In [103]:
tranzact_data = tranzact_data[tranzact_data['features'].str.contains(' ')]
tranzact_data = tranzact_data.reset_index(drop=True)
tranzact_data.count()


Company     27044
Supplier    27044
Item        27044
features    27044
dtype: int64

In [104]:
tranzact_data['features'][359]

'chromium manganese spring steel sp annealed grade sup9 st bar'

In [105]:
for i in range(len(tranzact_data)):
    if tranzact_data['features'][i] == '':
        
        tranzact_data['Supplier'][i] = np.nan
    
        
tranzact_data = tranzact_data.dropna()
tranzact_data = tranzact_data.reset_index(drop=True)
print(len(tranzact_data))

27044


In [106]:
## To check no. of Suppliers & no. of unique featuresin the data base :

print('Supplier = ', len(tranzact_data.Supplier.value_counts()))
print('Unique Features = ', len(tranzact_data.features.value_counts()))
print('Tranzact_data length = ', len(tranzact_data))
tranzact_data.tail()

Supplier =  2257
Unique Features =  15941
Tranzact_data length =  27044


,Company,Supplier,Item,features
27039,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'r',ferrul number
27040,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'y',ferrul number
27041,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'b',ferrul number
27042,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'n',ferrul number
27043,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long


In [7]:
## Above indicates that there are 2337 suppliers supplying 28921 items(as per no. of rows)
## above indicates that there are 16728 unique features out of 28921 items(as per rows)

In [8]:
## Let's reduce no. of Suppliers : criterian - apperaing at least 10 times in the data frame

# tranzact_data_short = tranzact_data.groupby("Supplier").filter(lambda x: len(x) > 10)

# print(len(tranzact_data_short))
# print(len(tranzact_data_short.Supplier.value_counts()))

In [9]:
## Above indicates that there are now 391 suppliers supplying 23649 items(as per no. of rows)

In [108]:
## Removing duplicate features + Supplier combinations :

for i in range (len(tranzact_data)):
    
    idx_feature = []
    idx_Supplier = []
    idx_common = []

    idx_feature = tranzact_data.index[tranzact_data['features'] == tranzact_data['features'][i]].tolist()
    idx_Supplier = tranzact_data.index[tranzact_data['Supplier'] == tranzact_data['Supplier'][i]].tolist()
    idx_common = list(set(idx_feature).intersection(idx_Supplier))
    
    if len(idx_common) > 1:
        
        tranzact_data['Supplier'][idx_common[1:]] = np.nan
        
tranzact_data_final = tranzact_data.dropna()
tranzact_data_final = tranzact_data_final.reset_index(drop=True)
tranzact_data_final.head()
                           

,Company,Supplier,Item,features
0,M/S Plastech,Pukhraj Additives LLP,zincoplast zinc,zincoplast zinc
1,M/S Plastech,Pukhraj Additives LLP,zincoplast pyn,zincoplast pyn
2,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup...",chromium manganese spring steel rolled sup9 wi...
3,Modern Engineering & Spring Company,Jay Khodiyar Steel & Metals,ms sheet 8 ft x 4 ft x 1.00 mm - 4 nos,ms sheet
4,Modern Engineering & Spring Company,Dynachem Engineers,ms plate 160 x 160 x 6 - 4 holes drg. z-413146...,ms plate holes drg rev ae0


In [109]:
tranzact_data_final.tail()

,Company,Supplier,Item,features
18809,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg11,pvc gland pg11
18810,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg13.5,pvc gland pg13
18811,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland m25x1.5,pvc gland m25x1
18812,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'y',ferrul number
18813,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long


In [110]:
## To cross check no. of Suppliers & no. of unique features in the data base :

print('Supplier = ', len(tranzact_data_final.Supplier.value_counts()))
print('Unique Features = ', len(tranzact_data_final.features.value_counts()))
print('Total no. of entrys = ', len(tranzact_data_final))
tranzact_data_final.tail()

Supplier =  2257
Unique Features =  15941
Total no. of entrys =  18814


,Company,Supplier,Item,features
18809,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg11,pvc gland pg11
18810,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg13.5,pvc gland pg13
18811,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland m25x1.5,pvc gland m25x1
18812,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'y',ferrul number
18813,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long


In [111]:
# Creating seacrh engine file :

col = ['Supplier', 'features']
tranzact_data_search = tranzact_data_final[col]


tranzact_data_search['Supplier_id'] = tranzact_data_search['Supplier'].factorize()[0]  ## Indexing

supplier_id_tranzact_data_search = tranzact_data_search[['Supplier', 'Supplier_id']].drop_duplicates(). \
                                                        sort_values('Supplier_id')

## Create two dictionaries :

supplier_to_id = dict(supplier_id_tranzact_data_search.values)
id_to_supplier = dict(supplier_id_tranzact_data_search[['Supplier_id', 'Supplier']].values)
tranzact_data_search.tail()

,Supplier,features,Supplier_id
18809,M/S. S. K. Enterprise,pvc gland pg11,308
18810,M/S. S. K. Enterprise,pvc gland pg13,308
18811,M/S. S. K. Enterprise,pvc gland m25x1,308
18812,M/S. S. K. Enterprise,ferrul number,308
18813,Anant Traders,hard copper pipe gauge long,2060


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = ['This is the first document.', \
          'This document is the second document.', \
          'And this is the third one.', \
          'Is this the first document?']

vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', ngram_range=(1, 2))

X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X.shape)

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X.shape)

['and', 'and this', 'document', 'document is', 'first', 'first document', 'is', 'is the', 'is this', 'one', 'second', 'second document', 'the', 'the first', 'the second', 'the third', 'third', 'third one', 'this', 'this document', 'this is', 'this the']
(4, 22)
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
(4, 9)


In [112]:
# Digitising Process No. 1

from sklearn.feature_extraction.text import TfidfVectorizer

# create the transform
my_vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', ngram_range=(1, 2))

# tokenize and build vocab
digital_features = my_vectorizer.fit(tranzact_data_search['features'])

# encode document
my_vector = digital_features.transform(tranzact_data_search['features']).toarray()
print(type(my_vector))
print(my_vector.shape)

<class 'numpy.ndarray'>
(18814, 48453)


In [113]:
## Input Text data :

text_data = ['housing casting smh500 s.g. iron a00000285']

In [114]:
## From Teaxt data to text feature :

text_data1 = pd.Series(text_data)

pd.options.mode.chained_assignment = None

text_data_feature = text_data1.str.replace(r'\b\d+\b','')  ## remove integers

text_data_feature = text_data_feature.str.replace(r'\W',' ')   ## remove puntuations

text_data_feature = text_data_feature.str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

text_data_feature = text_data_feature.str.replace(r'\b\w\b','') ## remove stand alone single letters

text_data_feature = text_data_feature.str.replace(r'\s+',' ') ## remove gaps between words to singe gap


## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")

words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')

text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())

text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
                                                                                key=x.split().index))]).lower())
print(text_data_feature)
print(len(text_data_feature))

0    housing casting smh500 iron a00000285
dtype: object
1


In [115]:
def cos_cdist(matrix, vector):
    """
    Compute the cosine distances between each row of matrix and vector.
    """
    v = vector.reshape(1, -1)
    return scipy.spatial.distance.cdist(matrix, v, 'cosine').reshape(-1)

In [116]:
## Creating test vector from Text features :

merge_text_feature = []
merge_text_feature_digit = []
merge_digit_vector = []
test_digit_vector = []

merge_text_feature = tranzact_data_search['features'].append(pd.Series(text_data_feature))

merge_text_feature = merge_text_feature.reset_index(drop=True)

merge_text_feature_digit = my_vectorizer.fit(merge_text_feature)

# encode document
merge_digit_vector = merge_text_feature_digit.transform(merge_text_feature).toarray()


test_digit_vector = merge_digit_vector[len(merge_digit_vector)-1].reshape((merge_digit_vector.shape)[1],)

print(test_digit_vector.shape)

## Finding cosine similarity :

c_d = []

c_d = np.round((1 - cos_cdist(my_vector, test_digit_vector)), 3)

print(c_d)
print(c_d.shape)
print(type(c_d))
print(max(c_d))

(48453,)
[0. 0. 0. ... 0. 0. 0.]
(18814,)
<class 'numpy.ndarray'>
1.0


In [58]:
print(tranzact_data_search['features'][85])

copper pipe


In [117]:
col = ['Supplier', 'features']
text_data_search_result = []



In [118]:
max_c_s_index = []
max_c_s_index = np.argsort(c_d)[-10:][::-1]
print(max_c_s_index)
print(text_data_feature)

for i in max_c_s_index:
    print(c_d[i])
#    print(tranzact_data_final['Item'][i])
    print(tranzact_data_search['features'][i])
    print(tranzact_data_search['Supplier'][i])


[ 2305  3439  1977  1250  1914  1325  2296 16319  1329  1231]
0    housing casting smh500 iron a00000285
dtype: object
1.0
housing casting smh500 iron a00000285
Jay Metal Tech
0.409
worm gear casting smh500 iron astm a536 a00000290
Jay Metal Tech
0.324
housing casting km40
Bhagwati Casters Pvt. Ltd.
0.29
housing casting ra125 iron a00002120
Jmt India Inc
0.289
smh250 housing casting iron a00000407
Jay Metal Tech
0.287
housing casting ra250 iron a00002121
Jay Metal Tech
0.245
housing casting ra100 a00004596 astm a536 iron
Jay Metal Tech
0.227
smh case pipe casting smh500 is2062 e250a a00005207
MEGHA PATTERN WORKS
0.202
housing casting ra150 astm a536 a00002119
Jay Metal Tech
0.188
housing casting km55 is210 gr fg260 cast iron astm a126 class kma24632
Bhagwati Casters Pvt. Ltd.


In [119]:
result = []
result_frame = []
max_c_s_index = []
max_c_s_index = np.argsort(c_d)[-50:][::-1]
print(max_c_s_index)
print(text_data_feature)

for i in max_c_s_index:
    
#    d = {'c_index':[c_d]}
#    df2 = pd.DataFrame(np.array([c_d[i], tranzact_data_search['features'][i], tranzact_data_search['Supplier'][i]]), \
#                      columns=['c_index', 'features', 'Supplier'])
    result.append(np.array([c_d[i], tranzact_data_final['Item'][i], tranzact_data_search['Supplier'][i]]))
#    text_data_search_result.append(c_d[i])
#    print(tranzact_data_final['Item'][i])
#    print(tranzact_data_search['features'][i])
#    print(tranzact_data_search['Supplier'][i])

result_frame = pd.DataFrame(result, columns=['c_index', 'features', 'Supplier'])
#print(result_frame)


[ 2305  3439  1977  1250  1914  1325  2296 16319  1329  1231  1979  1326
  2139  1343  5682  7818 14555  1585  1150  1154  2260 13240  1299   361
 14535 16341   568  8174 12909  2273  7123  1913  5616   525 14536  1393
  3434  1692  7410  1233 17330 12851  5424  8779 16792   736   735 16320
  1169 10700]
0    housing casting smh500 iron a00000285
dtype: object


In [120]:
zz = max(result_frame['c_index'])
print(zz)
result_frame.info()

1.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
c_index     50 non-null object
features    50 non-null object
Supplier    50 non-null object
dtypes: object(3)
memory usage: 1.2+ KB


In [121]:
result_frame.c_index = result_frame.c_index.astype('float64')
result_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
c_index     50 non-null float64
features    50 non-null object
Supplier    50 non-null object
dtypes: float64(1), object(2)
memory usage: 1.2+ KB


In [122]:
result_frame.drop_duplicates(subset='Supplier', keep = 'first', inplace = True)
result_frame = result_frame.reset_index(drop=True)
print(result_frame)

    c_index                                           features  \
0     1.000        housing casting smh500  s.g iron  a00000285   
1     0.324                               housing casting km40   
2     0.290         housing casting ra125  s.g iron  a00002120   
3     0.227  smh case pipe casting smh500 (is2062 e250a) a0...   
4     0.137                            spring housing 10" #300   
5     0.137                            spring housing 10" #300   
6     0.127                                        housing tpw   
7     0.106  82mm x 26mm transperent dust cover sticker smh...   
8     0.102                                 soldering iron 25w   
9     0.102                                 soldering iron 25w   
10    0.101                               soldering iron stand   
11    0.099                               body casting 8" #150   
12    0.099                              body casting 24" #900   
13    0.098                             soldering iron 25 watt   
14    0.09

In [123]:
kount = 0


for i in range(len(result_frame['Supplier'])):
    if result_frame['c_index'][i] >= 0.10:
        print(result_frame['Supplier'][i])
        kount = kount+1
        if kount == 5 :
            break
    


Jay Metal Tech
Bhagwati Casters Pvt. Ltd.
Jmt India Inc
MEGHA PATTERN WORKS
CESARE BONETTI INDIA PVT. LTD.


In [137]:
## Preparing Supplier_id + max cosine value matrix for the text vector :

max_c_s_index = []
max_c_s_index = np.argsort(c_d)[-10:][::-1]
print(max_c_s_index)

for i in max_c_sindex:
    print(c_d[i])

search_supplier_id = []
search_supplier_id_mod= []
cosine_id_matrix = []
cosine_id_matrix_mod = []


for i in (max_c_s_index):
    search_supplier_id.append(tranzact_data_search['Supplier_id'][i])
search_supplier_id_mod = np.array(search_supplier_id)
print(search_supplier_id_mod)

cosine_id_matrix = np.vstack((search_supplier_id_mod, max_c_s_index)).T
print(cosine_id_matrix)

cosine_id_matrix_mod = cosine_id_matrix[np.unique(cosine_id_matrix[:, 0], return_index=True)[1]]

print(cosine_id_matrix_mod)

print(min(c_d))
print(type(cosine_id_matrix_mod))
print(cosine_id_matrix_mod.shape)
print(len(cosine_id_matrix_mod[:,0]))


[19760 14776 14775   503    85 18364 18358 15370 19711 11585]
[2135  108  108   39   39 2135 2135 1781  610  311]
[[ 2135 19760]
 [  108 14776]
 [  108 14775]
 [   39   503]
 [   39    85]
 [ 2135 18364]
 [ 2135 18358]
 [ 1781 15370]
 [  610 19711]
 [  311 11585]]
[[   39   503]
 [  108 14776]
 [  311 11585]
 [  610 19711]
 [ 1781 15370]
 [ 2135 19760]]
0.0
<class 'numpy.ndarray'>
(6, 2)
6


In [133]:
import numpy_indexed as npi

column_to_groupby = 1
groups, reduced = npi.group_by(cosine_id_matrix[:,column_to_groupby]).min(cosine_id_matrix)
print(reduced)

[[   85    39]
 [14775   108]
 [11585   311]
 [19711   610]
 [15370  1781]
 [18358  2135]]


In [119]:
## Searching :

if len(cosine_id_matrix_mod[:,0]) > 0:
    kount = 0
    for k in (cosine_id_matrix_mod[:, 1]):
                   print(tranzact_data_search['Supplier'][k])
                   print(tranzact_data_search['features'][k]) 
                   kount = kount + 1
                   if kount == 5:
                       break
                   
print("There are no suppliers for this itemn")
    
    



#max_c_s_index = np.argsort(c_d)[-10:][::-1]

#for k in range(len(max_c_s_index)):
    
#    if tranzact_data_search['Supplier_id'][max_c_s_index[k]] \
#                 == tranzact_data_search['Supplier_id'][max_c_s_index[k+1]]:
        
#        print(max_c_s_index[k])
#        max_c_s_index_1 = np.delete(max_c_s_index, k)
#        max_c_s_index = max_c_s_index.reset_index(drop=True)
 #       print(max_c_s_index)
#                
#    if k == len(max_c_s_index)-2:
        
#        break
        
#for k in range(len(max_c_s_index_1)):
    
#    kount = 0

#    if c_d[max_c_s_index_1[0]] < 0.3:
    
#        print('There are No suppliers for this item')
    
#    else:
    
#        for i in range(len(max_c_s_index_1)):
        
#            if c_d[max_c_s_index_1[i]] < 0.3:
            
#                print('There are No more Suppliers')
                
#                break
            
#            print(tranzact_data_search['Supplier'][max_c_s_index_1[i]])
#            print(tranzact_data_search['Supplier_id'][max_c_s_index_1[i]])
#            #print(tranzact_data_final['Item'][max_c_s_index[i]])
#            kount = kount+1
            
#            if kount == 10 :
#                break



Shanti Steel Industries
chromium manganese spring steel sp annealed sup9 wire rod
Mouser Electronics Inc.
ic 74hc595 dip
Mouser Electronics Inc.
zener diodes 5v 500mw make semiconductor manufacturer part mmsz5222bt1g
VSN ELECTRO COMPONENTS PVT LTD
lcd display yellow_green sdcb1602 01d make sinda
Amazon.in
plastic electronics screwdriver set make bosch part number
There are no suppliers for this itemn
